<a href="https://colab.research.google.com/github/anassaffo/Leupahan_University_Projects/blob/main/simulated_annealing_and_local_search_Simualted_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Packages

In [ ]:
from random import randint
import random
from math import exp
from math import log
import pdb
import random 

# Defining Simualted Annealing

In [ ]:
class minimize():
    '''Simple Simulated Annealing
    '''

    def __init__(self, func, x0, opt_mode, find_neigbour_solution, cooling_schedule='linear', step_max=1000, t_min=0, t_max=100, bounds=[], alpha=None, damping=1):

        # checks
        assert opt_mode in ['combinatorial','continuous'], 'opt_mode must be either "combinatorial" or "continuous"'
        assert cooling_schedule in ['linear','exponential','logarithmic', 'quadratic'], 'cooling_schedule must be either "linear", "exponential", "logarithmic", or "quadratic"'

        self.find_neigbour_solution = find_neigbour_solution

        # initialize starting conditions
        self.t = t_max
        self.t_max = t_max
        self.t_min = t_min
        self.step_max = step_max
        self.opt_mode = opt_mode
        self.hist = []
        self.cooling_schedule = cooling_schedule

        self.cost_func = func
        self.x0 = x0
        self.bounds = bounds[:]
        self.damping = damping
        self.current_state = self.x0
        self.current_energy = func(self.x0)
        self.best_state = self.current_state
        self.best_energy = self.current_energy


        # initialize optimization scheme
        if self.opt_mode == 'combinatorial': self.get_neighbor = self.move_combinatorial
        if self.opt_mode == 'continuous': self.get_neighbor = self.move_continuous


        # initialize cooling schedule
        if self.cooling_schedule == 'linear':
            if alpha != None:
                self.update_t = self.cooling_linear_m
                self.cooling_schedule = 'linear multiplicative cooling'
                self.alpha = alpha

            if alpha == None:
                self.update_t = self.cooling_linear_a
                self.cooling_schedule = 'linear additive cooling'

        if self.cooling_schedule == 'quadratic':
            if alpha != None:
                self.update_t = self.cooling_quadratic_m
                self.cooling_schedule = 'quadratic multiplicative cooling'
                self.alpha = alpha

            if alpha == None:
                self.update_t = self.cooling_quadratic_a
                self.cooling_schedule = 'quadratic additive cooling'

        if self.cooling_schedule == 'exponential':
            if alpha == None: self.alpha =  0.8
            else: self.alpha = alpha
            self.update_t = self.cooling_exponential

        if self.cooling_schedule == 'logarithmic':
            if alpha == None: self.alpha =  0.8
            else: self.alpha = alpha
            self.update_t = self.cooling_logarithmic


        # begin optimizing
        self.step, self.accept = 1, 0
        while self.step < self.step_max and self.t >= self.t_min and self.t>0:
            # get neighbor
            proposed_neighbor = self.get_neighbor()

            # check energy level of neighbor
            E_n = self.cost_func(proposed_neighbor)
            dE = -(E_n - self.current_energy)
                       
            # pdb.set_trace()

            # determine if we should accept the current neighbor
            if random.random() < self.safe_exp(-dE / self.t):
                self.current_energy = E_n
                self.current_state = proposed_neighbor[:]
                self.accept += 1

            # check if the current neighbor is best solution so far
            if E_n > self.best_energy:
                self.best_energy = E_n
                self.best_state = proposed_neighbor[:]

            # persist some info for later
            self.hist.append([
                self.step,
                self.t,
                self.current_energy,
                self.best_energy])

            # update some stuff
            self.t = self.update_t(self.step)
            self.step += 1

        # generate some final stats
        self.acceptance_rate = self.accept / self.step


    def move_continuous(self):
        # preturb current state by a random amount
        neighbor = [item + ((random() - 0.5) * self.damping) for item in self.current_state]

        # clip to upper and lower bounds
        if self.bounds:
            for i in range(len(neighbor)):
                x_min, x_max = self.bounds[i]
                neighbor[i] = min(max(neighbor[i], x_min), x_max)

        return neighbor


    def move_combinatorial(self):
        '''Swaps two random nodes along path
        Not the most efficient, but it does the job...
        '''
        # p0 = randint(0, len(self.current_state)-1)
        # p1 = randint(0, len(self.current_state)-1)

        # neighbor = self.current_state[:]
        # neighbor[p0], neighbor[p1] = neighbor[p1], neighbor[p0]

        neighbor = find_neigbour_solution(self.current_state)

        return neighbor


    def results(self):
        print('+------------------------ RESULTS -------------------------+\n')
        print(f'      opt.mode: {self.opt_mode}')
        print(f'cooling sched.: {self.cooling_schedule}')
        if self.damping != 1: print(f'       damping: {self.damping}\n')
        else: print('\n')

        print(f'  initial temp: {self.t_max}')
        print(f'    final temp: {self.t:0.6f}')
        print(f'     max steps: {self.step_max}')
        print(f'    final step: {self.step}\n')
        print(f'    final solution: {self.best_state}\n')

        print(f'  final energy: {self.best_energy:0.6f}\n')
        print('+-------------------------- END ---------------------------+')

    # linear multiplicative cooling
    def cooling_linear_m(self, step):
        return self.t_max /  (1 + self.alpha * step)

    # linear additive cooling
    def cooling_linear_a(self, step):
        return self.t_min + (self.t_max - self.t_min) * ((self.step_max - step)/self.step_max)

    # quadratic multiplicative cooling
    def cooling_quadratic_m(self, step):
        return self.t_min / (1 + self.alpha * step**2)

    # quadratic additive cooling
    def cooling_quadratic_a(self, step):
        return self.t_min + (self.t_max - self.t_min) * ((self.step_max - step)/self.step_max)**2

    # exponential multiplicative cooling
    def cooling_exponential_m(self, step):
        return self.t_max * self.alpha**step

    # logarithmical multiplicative cooling
    def cooling_logarithmic_m(self, step):
        return self.t_max / (self.alpha * log(step + 1))


    def safe_exp(self, x):
        try: return exp(x)
        except: return 0

# Defining Local Search:

In [ ]:
class LocalSearch():
    '''Simple Local Search
    '''

    def __init__(self, func, x0, find_neigbour_solution, step_max=1000):

        
        self.find_neigbour_solution = find_neigbour_solution

        # initialize starting conditions
    
        self.step_max = step_max

        self.cost_func = func
        self.get_neighbor = self.move_combinatorial
        self.x0 = x0
        
        self.current_state = self.x0
        self.current_energy = func(self.x0)
        self.best_state = self.current_state
        self.best_energy = self.current_energy

        # begin optimizing
        self.step, self.accept = 1, 0
        while self.step < self.step_max:
            # get neighbor
            proposed_neighbor = self.get_neighbor()

            # check energy level of neighbor
            E_n = self.cost_func(proposed_neighbor)
            dE = -(E_n - self.current_energy)
                       

            # determine if we should accept the current neighbor
            if dE > 0:
                self.current_energy = E_n
                self.current_state = proposed_neighbor[:]
                self.accept += 1

            # check if the current neighbor is best solution so far
            if E_n > self.best_energy:
                self.best_energy = E_n
                self.best_state = proposed_neighbor[:]

            # update some stuff
            self.step += 1

        # generate some final stats
        self.acceptance_rate = self.accept / self.step


    def move_combinatorial(self):
        neighbor = find_neigbour_solution(self.current_state)
        return neighbor


    def results(self):
        print('+------------------------ RESULTS -------------------------+\n')
  
        print(f'    max steps: {self.step_max}')
        print(f'    final step: {self.step}\n')
        print(f'    final solution: {self.best_state}\n')

        print(f'  final energy: {self.best_energy:0.6f}\n')
        print('+-------------------------- END ---------------------------+')


# Functions: 

In [ ]:
def get_solution_weight(solution):
  return sum([data[item]["wt"] for item in solution])

def get_solution_value(solution):
  return sum([data[item]["val"] for item in solution])

def gen_init_solution(data):
  solution_0 = []
  total_w = 0
  # get the items in the data:
  items = list(data.keys())
  # weight constraint:
  while total_w <= max_w:
    # select a random item:
    selected_item = random.choice(items)
    # get the weight from the data:
    item_weight = data[selected_item]["wt"]
    # keeping the weight constraint with update:
    if total_w + item_weight <= max_w:
      total_w += item_weight
      solution_0 += [selected_item]
      items.remove(selected_item)
    else:
      break
  return solution_0

def find_neigbour_solution(solution):
  solution = solution.copy()
  # your data without the solution:
  remaining = list(set(data.keys()) - set(solution)) 
  solution_weight = get_solution_weight(solution)

  # randomly select an item in the solution to remove:
  to_remove = random.choice(solution) 
  to_remove_weight = get_solution_weight([to_remove])
  # randomly select an item in the solution to add:
  to_add = random.choice(remaining) 
  to_add_weight = get_solution_weight([to_add])
  # weight constraint with update:
  if solution_weight + to_add_weight - to_remove_weight <= max_w and to_add not in solution:
    solution_weight += to_add_weight - to_remove_weight
    solution.remove(to_remove) 
    solution += [to_add]

  # weight constraint:
  if solution_weight <= max_w:
    # add a random item in the remaining:
    to_add = random.choice(remaining)
    to_add_weight = get_solution_weight([to_add])
    # repeat the weight constraint and update:
    if solution_weight + to_add_weight <= max_w and to_add not in solution:
      solution_weight += to_add_weight
      solution += [to_add]

  return solution

# Simualted Data: (Knapsack Problem)

In [ ]:
# values of each item:
val = [60, 121, 120, 23, 123, 54]
# weights of each item:
wt = [10, 20, 30, 43, 4, 12]
# the constraints:
max_w = 50
n = len(val)
# create the dataset:
data = {idx: {"val": v, "wt": w} for idx, (v, w) in enumerate(zip(val, wt))}
data


{0: {'val': 60, 'wt': 10},
 1: {'val': 121, 'wt': 20},
 2: {'val': 120, 'wt': 30},
 3: {'val': 23, 'wt': 43},
 4: {'val': 123, 'wt': 4},
 5: {'val': 54, 'wt': 12}}

## find an initial solution:

In [ ]:
solution_0 = gen_init_solution(data)
print(solution_0, get_solution_value(solution))

[1, 0, 4] 241


## find a neighbour solution:

In [ ]:
solution = find_neigbour_solution(solution_0)
print(solution, get_solution_value(solution))


[1, 0, 4] 304


## Simulated Annealing:

In [ ]:
sa_minimizer = minimize(get_solution_value,
                        solution, opt_mode="combinatorial", find_neigbour_solution=find_neigbour_solution,
                        step_max=10000, t_min=0, t_max=1000, bounds=[], alpha=None, damping=1)
sa_minimizer.results()

+------------------------ RESULTS -------------------------+

      opt.mode: combinatorial
cooling sched.: linear additive cooling


  initial temp: 1000
    final temp: 0.100000
     max steps: 10000
    final step: 10000

    final solution: [4, 5, 0, 1]

  final energy: 358.000000

+-------------------------- END ---------------------------+


## local Search:

In [ ]:
solution = gen_init_solution(data)
local_search_minimizer = LocalSearch(get_solution_value,
                        solution, find_neigbour_solution=find_neigbour_solution,
                        step_max=10000)
local_search_minimizer.results()

+------------------------ RESULTS -------------------------+

    max steps: 10000
    final step: 10000

    final solution: [0, 4, 1]

  final energy: 304.000000

+-------------------------- END ---------------------------+


## Itterative Local Search:

In [ ]:
# generate multiple solutions:
solution = gen_init_solution(data)
solution = find_neigbour_solution(solution)
solution = find_neigbour_solution(solution)
local_search_minimizer = LocalSearch(get_solution_value,
                        solution, find_neigbour_solution=find_neigbour_solution,
                        step_max=10000)
local_search_minimizer.results()

+------------------------ RESULTS -------------------------+

    max steps: 10000
    final step: 10000

    final solution: [4, 0, 1, 5]

  final energy: 358.000000

+-------------------------- END ---------------------------+
